In [1]:
#订购方案

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xlrd
import xlwt
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
from scipy.stats import zscore

In [2]:
# 打开文件
xlsx = xlrd.open_workbook('附件1 近5年402家供应商的相关数据.xlsx')
# 查看所有sheet列表
print('All sheets: %s' % xlsx.sheet_names())
sheet2 = xlsx.sheets()[1]    # 获得第1张sheet，索引从0开始
sheet2_name = sheet2.name    # 获得名称
sheet2_cols = sheet2.ncols   # 获得列数
sheet2_nrows = sheet2.nrows  # 获得行数
print('Sheet2 Name: %s\nSheet2 cols: %s\nSheet1 rows: %s' % (sheet2_name, sheet2_cols, sheet2_nrows))


All sheets: ['企业的订货量（m³）', '供应商的供货量（m³）']
Sheet2 Name: 供应商的供货量（m³）
Sheet2 cols: 242
Sheet1 rows: 403


In [3]:
dic_g={}

In [4]:
for i in range(sheet2.nrows-1):
            lis_1=[]
            sum_g=0
            
            for j in range(2,sheet2.ncols):
                lis_1.append(sheet2.cell(i+1,j).value)
                
            dic_g[sheet2.cell(i+1,0).value]=lis_1
           

In [5]:
dic_gong=list(dic_g.values())

In [6]:
dic_gg={}
for i in range(1,402):
    dic_gg[i]=dic_gong[i]

In [7]:
xlsx_1 = xlrd.open_workbook('排名.xlsx')
# 查看所有sheet列表
print('All sheets: %s' % xlsx_1.sheet_names())

All sheets: ['指标分析', 'Sheet1', 'Sheet2']


In [8]:
sheet1 = xlsx_1.sheets()[2]    # 获得第1张sheet，索引从0开始
sheet1_name = sheet1.name    # 获得名称
sheet1_cols = sheet1.ncols   # 获得列数
sheet1_nrows = sheet1.nrows  # 获得行数
print('Sheet1 Name: %s\nSheet1 cols: %s\nSheet1 rows: %s' % (sheet1_name, sheet1_cols, sheet1_nrows))


Sheet1 Name: Sheet2
Sheet1 cols: 1
Sheet1 rows: 51


In [9]:
paiming=[]
for i in range(1,sheet1.nrows):
       paiming.append(int(sheet1.cell(i,0).value))  
                
            

In [10]:
list_type=[]
for j in range(1,sheet2.nrows):
    list_type.append(sheet2.cell(j,1).value)

In [11]:
gong_si=[]
gong_t=[]


In [12]:
for i in range(len(paiming)):
    for key in dic_gg:
        #print(key)
        if key==paiming[i]:
            #print(dic_gg[key-1])
            gong_si.append(dic_gg[key-1])
            gong_t.append(list_type[key-1])

In [13]:
first_50=pd.DataFrame(gong_si)
first_50_t=pd.DataFrame(gong_t)

In [14]:
first_50.to_excel("3.xlsx", sheet_name="排名")


In [15]:
first_50_t.to_excel("4.xlsx", sheet_name="type")


In [99]:
#给前50名商家做预测
def GM11(x, n):
    '''
    灰色预测
    x：序列，numpy对象
    n:需要往后预测的个数
    '''
    x1 = x.cumsum()  # 一次累加
    z1 = (x1[:len(x1) - 1] + x1[1:]) / 2.0  # 紧邻均值
    z1 = z1.reshape((len(z1), 1))
    B = np.append(-z1, np.ones_like(z1), axis=1)
    Y = x[1:].reshape((len(x) - 1, 1))
    # a为发展系数 b为灰色作用量
    [[a], [b]] = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)  # 计算参数
    result = (x[0] - b / a) * np.exp(-a * (n - 1)) - (x[0] - b / a) * np.exp(-a * (n - 2))
    S1_2 = x.var()  # 原序列方差
    e = list()  # 残差序列
    for index in range(1, x.shape[0] + 1):
        predict = (x[0] - b / a) * np.exp(-a * (index - 1)) - (x[0] - b / a) * np.exp(-a * (index - 2))
        e.append(x[index - 1] - predict)
    S2_2 = np.array(e).var()  # 残差方差
    C = S2_2 / S1_2  # 后验差比
    print(C)
    if C <= 0.35:
        assess = '后验差比<=0.35，模型精度等级为好'
    elif C <= 0.5:
        assess = '后验差比<=0.5，模型精度等级为合格'
    elif C <= 0.65:
        assess = '后验差比<=0.65，模型精度等级为勉强'
    else:
        assess = '后验差比>0.65，模型精度等级为不合格'
    # 预测数据
    predict = list()
    for index in range(x.shape[0] + 1, x.shape[0] + n + 1):
        predict.append((x[0] - b / a) * np.exp(-a * (index - 1)) - (x[0] - b / a) * np.exp(-a * (index - 2)))
    predict = np.array(predict)
    return {
        'a': {'value': a, 'desc': '发展系数'},
        'b': {'value': b, 'desc': '灰色作用量'},
        'predict': {'value': result, 'desc': '第%d个预测值' % n},
        'C': {'value': C, 'desc': assess},
        'predict': {'value': predict, 'desc': '往后预测%d个的序列' % (n)},
    }


In [101]:
if __name__ == '__main__':
    xlsx = xlrd.open_workbook('每年最大值-50.xlsx')
    # 查看所有sheet列表
    print('All sheets: %s' % xlsx.sheet_names())
    sheet3 = xlsx.sheets()[0]    # 获得第1张sheet，索引从0开始
    dic={}
    for i in range(sheet3.nrows-1):
            lis=[]
            for j in range(2,sheet3.ncols):
                lis.append(sheet3.cell(i+1,j).value)
            dic[sheet3.cell(i+1,0).value]=lis
    dic_50=list(dic.values())
    re=[]
    for i in range (0,50):
        S1=dic_50[i]
        data = np.array(S1)
        x = data[6:11]# 输入数据
        y = range(1) # 需要预测的数据
        result = GM11(x, len(y))
        predict = result['predict']['value']
        if (predict<=0):
            predict=[0.0]
        else:
            predict = np.round(predict, 0)
        re.append(predict)
        #predict=DataFrame(predict)#这时候是以行为标准写入的
        #predict=predict.T#转置之后得到想要的结果

        print('预测值:', predict)
       

All sheets: ['Sheet1']
0.06225431220384459
预测值: [9643.]
0.35687113416111754
预测值: [2089.]
0.179915531248323
预测值: [1549.]
0.12422033379323778
预测值: [4300.]
0.13161365554658724
预测值: [625.]
0.10072435446541439
预测值: [1552.]
0.0684767768458593
预测值: [2064.]
2.1228163830848996
预测值: [3550.]
0.22334883921602408
预测值: [8224.]
0.08156303940836533
预测值: [1022.]
0.11329515604261568
预测值: [672.]
0.04179602547514522
预测值: [68296.]
0.04203298847595191
预测值: [84097.]
0.33319510476616226
预测值: [1061.]
0.05509973937706734
预测值: [1932.]
0.0588924416684229
预测值: [605.]
0.07880560409423668
预测值: [437.]
0.06258337043463567
预测值: [44048.]
0.08247069178354317
预测值: [471.]
0.052154782930028586
预测值: [1501.]
0.04575066905230675
预测值: [15158.]
0.08514449732743666
预测值: [12192.]
0.5072813844114191
预测值: [121.]
0.08314461080421064
预测值: [7720.]
0.12564722925154717
预测值: [326.]
0.22979395114286622
预测值: [346.]
0.03787193401268326
预测值: [459324.]
0.29834441099039327
预测值: [207.]
0.9654302663322382
预测值: [214.]
0.4372460909309673
预测值: [215.

LinAlgError: Singular matrix

In [102]:
ll=[]
for i in range(len(re)):
    ll.append(list(re[i]))

In [103]:
ll

[[9643.0],
 [2089.0],
 [1549.0],
 [4300.0],
 [625.0],
 [1552.0],
 [2064.0],
 [3550.0],
 [8224.0],
 [1022.0],
 [672.0],
 [68296.0],
 [84097.0],
 [1061.0],
 [1932.0],
 [605.0],
 [437.0],
 [44048.0],
 [471.0],
 [1501.0],
 [15158.0],
 [12192.0],
 [121.0],
 [7720.0],
 [326.0],
 [346.0],
 [459324.0],
 [207.0],
 [214.0],
 [215.0],
 [378.0],
 [1577.0],
 [348.0],
 [121.0],
 [0.0],
 [1093.0],
 [195.0],
 [128.0],
 [128.0],
 [1906.0],
 [343.0],
 [258.0],
 [127.0],
 [4.0],
 [11.0],
 [8.0]]

In [104]:
ll=pd.DataFrame(ll)

In [105]:
ll.to_excel("ll.xlsx", sheet_name="5")

In [33]:
re=pd.DataFrame(re)

In [35]:
re.to_excel("前50家24周预测.xlsx", sheet_name="5")